In [21]:
%load_ext autoreload
%autoreload 2

import torch
import project_path
from training import *
from data_utils import get_data_loaders

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
NUM_CLIENTS = 5
ACCURACY_THRESHOLD = 93

In [25]:
#Load data
train_loaders, _, test_loader = get_data_loaders(BATCH_SIZE, NUM_CLIENTS, percentage_val=0.2, iid_split=False)
print(len(train_loaders))

#Initialize all clients
clients = []
for train_loader in train_loaders:
    client = Client(train_loader)
    clients.append(client)
torch.manual_seed(clients[0].seed)

* Using MNIST
** Reducing the data-set, (use --full for the full thing)
** Use 5000 train and 5000 test samples
5


In [26]:
testing_accuracy = 0
bits_transferred = 0
num_rounds = 0
bits_conserved = 0

centralServer = Client(test_loader)

while testing_accuracy < ACCURACY_THRESHOLD:
    num_rounds += 1
    print("Communication Round {0}".format(num_rounds))
    
    if num_rounds > 1:
        #Load server weights onto clients
        #TODO: Count number of transferred bits
        for client in clients:
            with torch.no_grad():
                client.model.load_state_dict(dict(centralServer.model.named_parameters()))
                
    #Perform E local training steps for each client
    for client_idx, client in enumerate(clients):
        print("Training client {0}".format(client_idx))
        for epoch in range(1, client.epochs + 1):
            train(client, epoch)

    #Average model parameters and test
    #TODO: Count number of transfered bits and sparsify gradients
    with torch.no_grad():
        dict_params = average_client_models(clients)
        centralServer.model.load_state_dict(dict_params)
    test_loss, testing_accuracy = test(centralServer)

#Save model
if centralServer.save_model:
     torch.save(centralServer.model.state_dict(), f"{centralServer.model_name}.pt")

Communication Round 1
Training client 0
Train Epoch: 1 Loss: 0.044400, Train accuracy: 97.70114942528735
Train Epoch: 2 Loss: 0.026691, Train accuracy: 98.74608150470219
Train Epoch: 3 Loss: 0.007348, Train accuracy: 99.16405433646813
Train Epoch: 4 Loss: 0.011483, Train accuracy: 99.68652037617555
Train Epoch: 5 Loss: 0.003100, Train accuracy: 100.0
Training client 1
Train Epoch: 1 Loss: 0.012312, Train accuracy: 99.4291151284491
Train Epoch: 2 Loss: 0.001269, Train accuracy: 99.61941008563274
Train Epoch: 3 Loss: 0.000596, Train accuracy: 99.80970504281636
Train Epoch: 4 Loss: 0.001286, Train accuracy: 100.0
Train Epoch: 5 Loss: 0.000018, Train accuracy: 100.0
Training client 2
Train Epoch: 1 Loss: 0.000000, Train accuracy: 99.8080614203455
Train Epoch: 2 Loss: 0.076870, Train accuracy: 99.61612284069098
Train Epoch: 3 Loss: 0.000004, Train accuracy: 99.90403071017275
Train Epoch: 4 Loss: 0.000086, Train accuracy: 100.0
Train Epoch: 5 Loss: 0.000000, Train accuracy: 100.0
Training cl

Train Epoch: 1 Loss: 0.129540, Train accuracy: 95.20897043832824
Train Epoch: 2 Loss: 0.158031, Train accuracy: 96.12640163098878
Train Epoch: 3 Loss: 0.103849, Train accuracy: 96.73802242609582
Train Epoch: 4 Loss: 0.052517, Train accuracy: 96.43221202854231
Train Epoch: 5 Loss: 0.015764, Train accuracy: 96.73802242609582
Training client 4
Train Epoch: 1 Loss: 0.058361, Train accuracy: 97.83281733746131
Train Epoch: 2 Loss: 0.064184, Train accuracy: 98.76160990712074
Train Epoch: 3 Loss: 0.020260, Train accuracy: 99.07120743034056
Train Epoch: 4 Loss: 0.014543, Train accuracy: 99.17440660474716
Train Epoch: 5 Loss: 0.102260, Train accuracy: 97.52321981424149
Test set: Average loss: 0.0264, Test accuracy: 3271 / 5000 (65%)

Communication Round 6
Training client 0
Train Epoch: 1 Loss: 0.171650, Train accuracy: 95.40229885057471
Train Epoch: 2 Loss: 0.066613, Train accuracy: 94.14838035527691
Train Epoch: 3 Loss: 0.027831, Train accuracy: 96.1337513061651
Train Epoch: 4 Loss: 0.040410, T

Train Epoch: 5 Loss: 0.037062, Train accuracy: 98.22361546499478
Training client 1
Train Epoch: 1 Loss: 0.000979, Train accuracy: 99.61941008563274
Train Epoch: 2 Loss: 0.001311, Train accuracy: 99.80970504281636
Train Epoch: 3 Loss: 0.000327, Train accuracy: 99.80970504281636
Train Epoch: 4 Loss: 0.003564, Train accuracy: 99.80970504281636
Train Epoch: 5 Loss: 0.006787, Train accuracy: 99.80970504281636
Training client 2
Train Epoch: 1 Loss: 0.019415, Train accuracy: 99.8080614203455
Train Epoch: 2 Loss: 0.004076, Train accuracy: 99.8080614203455
Train Epoch: 3 Loss: 0.002170, Train accuracy: 99.90403071017275
Train Epoch: 4 Loss: 0.002013, Train accuracy: 99.8080614203455
Train Epoch: 5 Loss: 0.002633, Train accuracy: 99.90403071017275
Training client 3
Train Epoch: 1 Loss: 0.084444, Train accuracy: 98.2670744138634
Train Epoch: 2 Loss: 0.045056, Train accuracy: 98.36901121304791
Train Epoch: 3 Loss: 0.124499, Train accuracy: 96.94189602446484
Train Epoch: 4 Loss: 0.058047, Train acc

Train Epoch: 5 Loss: 0.011101, Train accuracy: 99.28644240570846
Training client 4
Train Epoch: 1 Loss: 0.057298, Train accuracy: 99.27760577915377
Train Epoch: 2 Loss: 0.003469, Train accuracy: 99.48400412796698
Train Epoch: 3 Loss: 0.015953, Train accuracy: 99.8968008255934
Train Epoch: 4 Loss: 0.007988, Train accuracy: 99.58720330237358
Train Epoch: 5 Loss: 0.016469, Train accuracy: 99.8968008255934
Test set: Average loss: 0.0152, Test accuracy: 3816 / 5000 (76%)

Communication Round 15
Training client 0
Train Epoch: 1 Loss: 0.038467, Train accuracy: 93.31243469174504
Train Epoch: 2 Loss: 0.009773, Train accuracy: 98.43260188087774
Train Epoch: 3 Loss: 0.025895, Train accuracy: 98.53709508881923
Train Epoch: 4 Loss: 0.010914, Train accuracy: 98.64158829676072
Train Epoch: 5 Loss: 0.001483, Train accuracy: 98.85057471264368
Training client 1
Train Epoch: 1 Loss: 0.072770, Train accuracy: 95.71836346336822
Train Epoch: 2 Loss: 0.001184, Train accuracy: 99.80970504281636
Train Epoch: 3

Train Epoch: 4 Loss: 0.000077, Train accuracy: 100.0
Train Epoch: 5 Loss: 0.000104, Train accuracy: 100.0
Training client 2
Train Epoch: 1 Loss: 0.003214, Train accuracy: 99.90403071017275
Train Epoch: 2 Loss: 0.002337, Train accuracy: 99.90403071017275
Train Epoch: 3 Loss: 0.109276, Train accuracy: 99.8080614203455
Train Epoch: 4 Loss: 0.007779, Train accuracy: 99.90403071017275
Train Epoch: 5 Loss: 0.002531, Train accuracy: 99.90403071017275
Training client 3
Train Epoch: 1 Loss: 0.033444, Train accuracy: 99.08256880733946
Train Epoch: 2 Loss: 0.008208, Train accuracy: 98.87869520897044
Train Epoch: 3 Loss: 0.068299, Train accuracy: 98.36901121304791
Train Epoch: 4 Loss: 0.035421, Train accuracy: 98.57288481141693
Train Epoch: 5 Loss: 0.014675, Train accuracy: 99.69418960244649
Training client 4
Train Epoch: 1 Loss: 0.006597, Train accuracy: 99.79360165118679
Train Epoch: 2 Loss: 0.003055, Train accuracy: 99.8968008255934
Train Epoch: 3 Loss: 0.015711, Train accuracy: 99.896800825593

KeyboardInterrupt: 